<a href="https://colab.research.google.com/github/OmniDevsOficial/API-Censo-2022/blob/refactor%2Fmudan%C3%A7a-do-grafico-de-pizza/Backend/PlanejaSJC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bem-Vindo ao Planeja SJC!

Aqui, você encontrará uma ampla gama de recursos para explorar e compreender a dinâmica da nossa cidade. Nossa plataforma oferece mapas interativos que permitem visualizar diferentes aspectos do espaço urbano, desde a infraestrutura até as áreas verdes, ajudando você a entender melhor o ambiente em que vive. Além disso, disponibilizamos dados demográficos detalhados, que fornecem uma visão abrangente sobre a população, suas características e necessidades. Esses dados são essenciais para que cidadãos, planejadores urbanos e gestores públicos possam tomar decisões informadas e embasadas.

### Sumário dos Gráficos

1 - População Total por Bairro e Zona

2 - Crescimento Populacional por Zona entre 2010 e 2022

3 - Pirâmide Etária

4 - Densidade Populacional por Região

### Dependências do Projeto

In [1]:
import os
from contextlib import redirect_stdout
import io

# Define o diretório atual como o /content
os.chdir('/content')

# Baixa os arquivos do repositório Github para o /content do Colab
with redirect_stdout(io.StringIO()):
    !git clone https://github.com/OmniDevsOficial/API-Censo-2022.git

# Define a pasta onde estão os arquivos da base de dados
data_path = '/content/API-Censo-2022/Backend/database/'

# Realiza as importações para ler arquivos e mostrar gráficos
import pandas as pd
import plotly.express as px

### População Total por Bairro e Zona

In [7]:
#puxa a tabela de Pop por bairro e região do github e atribui as variaveis "Pop_Tabela_Bairro" e "Pop_Tabela_Regiao".
Pop_Tabela_Bairro= pd.read_excel(data_path + 'populacao_subsetores_completo.xlsx')

Pop_Tabela_Regiao= pd.read_excel(data_path + "censo_2022.xlsx", sheet_name="População por Região")

# Retira o Total Geral, que é referente a cidade e não uma região
Pop_Tabela_Regiao = Pop_Tabela_Regiao[Pop_Tabela_Regiao['Região'] != 'Total Geral']

#cria um gráfico em relação as tabelas, nesse caso a tabela censo_2022.
Pop_Total_Regiao=px.bar(Pop_Tabela_Regiao, x="Região", y="População", color="Região")
Pop_Total_Regiao.show()

Pop_Bairro_Centro=px.bar(Pop_Tabela_Bairro, x="Subsetor – Nome", y="População", color="Região",)
Pop_Bairro_Centro.show()

### Crescimento Populacional por Zona entre 2010 e 2022

In [3]:
# o colab faz a leitura da tabela e mostra um display dela
file_path = data_path + 'tabel_pop.xlsx'

df = pd.read_excel(file_path)
# Ordenar o DataFrame pela população de 2022
df_sorted = df.sort_values(by='Pop. 2022', ascending=False)

fig = px.bar(
    df_sorted,
    x='Região',
    y=['Pop. 2010', 'Pop. 2022'],
    barmode='group', # Agrupa as Colunas
    title='Crescimento Populacional por Região (2010 vs 2022)',
)

fig.update_layout(
    template='plotly_white',
    title_x=0.5,
    yaxis_title='Número de Habitantes',
    xaxis_title=None,
    legend_title='Ano do Censo'
)

fig.show()

#Aqui eu crio a varável de crescimento, leio o arquivo ja colocado anteriormente no códico e começo a estruturar o gráfico
crescimento=px.bar(df, x= "Região", y= "Crescimento %",
color="Região",
title= "Taxa de Crescimento Populacional"
)

#Essa parte serve pra estilizar o gráfico e seu layout
crescimento.update_layout(
    template='plotly_white',
    title_x=0.5,
    yaxis_title='Percentual de Habitantes',
    xaxis_title=None,
    legend_title='Regiões'
)

crescimento.show()

### Pirâmide Etária

In [4]:
faixa= pd.read_excel(data_path + 'etaria.xlsx')

display(faixa)
# Excluí a primeira linha - o cabeçalho. E a segunda linha - Onde mostra o total de habitantes da cidade.
faixa_sem_cabecalho_total = faixa.drop([0, 5])

fig = px.bar(
    faixa_sem_cabecalho_total,
    x = 'Grupos',
    y = ['2010','2022'])
             #labels={'variable': 'Anos e Porcentagem'})  Mudando o texto padrão do Plotly - Variable'
fig.update_layout(barmode='group')
fig.update_layout(
    legend_title_text='Anos',
    title='Grupos etários entre os Censos de 2010 e 2022',
    yaxis_title='População',
    xaxis_title=None)
fig.show()

,Grupos,2010,Porcentagem % 2010,2022,Porcentagem % 2022
0,Grupo etário,2010,2010.00,2022,2022.00
1,Crianças (0 a 14 anos),137245,0.22,128855,0.18
2,Jovens (15 a 29 anos),168655,0.27,144730,0.21
3,Adultos (30 a 64 anos),285169,0.45,342965,0.49
4,Idosos (65 anos ou mais),38852,0.06,80504,0.12
5,NaN,629921,1.00,697054,1.00


### Densidade Populacional por Região

In [5]:
excel_file = data_path + 'tabel_pop.xlsx'

df_pop = pd.read_excel(excel_file, sheet_name=0,  index_col=0,)

# Áreas(km²) das Regiões de SJC obtidas em: https://planodiretor.sjc.sp.gov.br/resources/uploads/EstudoTecnico/Anexo/PD_VOLUME_1_C7_REGIOES_URBANAS.pdf
areas = {
    "Norte": 63.73,
    "Sul": 56.51,
    "Leste": 134.69,
    "Oeste": 44.01,
    "Sudeste": 35.66,
    "Centro": 18.68,
    # "Rural": 745.7,
}
# Transforma o dicionário das áreas em um Dataframe
df_area = pd.DataFrame(list(areas.items()), columns=['Região', 'Área (km²)'])

# Une os dados de população e área
df = pd.merge(df_pop, df_area, on='Região')

# Retira os dados da Região Rural, pois possui densidade muito pequena para ser vista no gráfico
df = df[df['Região'] != 'Rural']

# Calcula a densidade populacional
df['2010'] = df['Pop. 2010'] / df['Área (km²)']
df['2022'] = df['Pop. 2022'] / df['Área (km²)']

# Ordena da maior densidade de 2022 para a menor
df_sorted = df.sort_values(by='2022', ascending=False)

# Organiza como mostrar e quais informações serão mostradas
fig = px.bar(
    df_sorted,
    x='Região',
    y=['2010', '2022'],
    barmode='group',
    title='Densidade Populacional por Região (2010 x 2022)',
    labels={'value': 'Densidade Populacional (hab/km²)', 'Região': 'Região', 'variable': 'Ano'},
)
fig.update_layout(legend_title_text='Ano', xaxis_title=None)

fig.show()